**Бонусы по категориям среднего чека** 

Алгоритм обрабатывает заказы в интернет-магазине и формирует отчет с категориями пользователей по средним чекам и как они пользуются бонусной системой.

Все конфиденциальные данные изменены.

In [1]:
# Импортируем нужные библиотеки и прочитаем датафрейм
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
orders = pd.read_excel('orders.xlsx')

In [2]:
# Проверяем количество дубликатов в датафрейме
print(f'Количество дубликатов: {orders.duplicated().sum()}')

Количество дубликатов: 0


In [3]:
# Смотрим количество пропущенных значений среди всех данных
orders.isna().sum()

order_number          0
status                0
user_id               0
company_id            0
total                 0
subtotal              0
pay_bonus           385
first_order_time      0
order_time            0
dtype: int64

In [4]:
# Заменим пропуски на нули и сделаем целые числа
orders['pay_bonus'] = orders['pay_bonus'].fillna(0)
orders['pay_bonus'] = orders['pay_bonus'].astype('int')

# Удалим нецелевые статусы из датафрейма
delete_status = ["A", "B", "D", "E", "F", "I", "U"]
orders = orders.query('status != @delete_status')

# Создадим столбцы с месяцем и годом по первому и настоящему заказу
orders['first_order_month'] = orders['first_order_time'].dt.month
orders['first_order_year'] = orders['first_order_time'].dt.year
orders['order_month'] = orders['order_time'].dt.month
orders['order_year'] = orders['order_time'].dt.year

# Создадим столбец с первым заказом
def FO(row):
    first = row['first_order_time']
    real = row['order_time']
    if first == real:
        return 1
    return 0
orders['FO'] = orders.apply(FO, axis=1)

# Создадим столбец с категоризацией чека
def total_category(row):
    total = row['total']
    if total < 1000:
        return "До 1000"
    if total <= 2000:
        return "1000-2000"
    if total <= 3000:
        return "2001-3000"
    if total <= 4000:
        return "3001-4000"
    if total <= 5000:
        return "4001-5000"
    if total <= 7000:
        return "5001-7000"
    if total <= 10000:
        return "7001-10000"
    return "Больше 10000"
orders['total_category'] = orders.apply(total_category, axis=1)

# Создадим столбец с указанием, пользуется бонусами или нет
def use_bonus(row):
    bonus = row['pay_bonus']
    if bonus == 0:
        return 0
    return 1
orders['use_bonus'] = orders.apply(use_bonus, axis=1)

In [5]:
# Проверим, чтобы были нужные статусы
orders['status'].unique()

array(['C', 'N', 'X', 'J', 'W'], dtype=object)

In [6]:
# Выведем первые 5 строк на экран
orders.head()

,order_number,status,user_id,company_id,total,subtotal,pay_bonus,first_order_time,order_time,first_order_month,first_order_year,order_month,order_year,FO,total_category,use_bonus
0,361051,C,587182,4,1700.00,1550.00,0,2019-12-16 21:52:15,2021-01-01 03:05:04,12,2019,1,2021,0,1000-2000,0
1,361052,C,468243,4,970.00,820.00,0,2018-08-18 11:59:53,2021-01-01 08:08:34,8,2018,1,2021,0,До 1000,0
2,361053,C,652508,4,1009.00,759.00,0,2021-01-01 08:35:33,2021-01-01 08:35:33,1,2021,1,2021,1,1000-2000,0
3,361054,C,623763,4,2035.00,1955.00,70,2020-08-07 16:32:44,2021-01-01 09:35:31,8,2020,1,2021,0,2001-3000,1
4,361055,C,608017,4,1266.00,1200.00,84,2020-04-25 19:49:44,2021-01-01 10:53:43,4,2020,1,2021,0,1000-2000,1


In [7]:
# Сделаем отчет за 2021 год
user_group_2021 = (
    orders.query('order_year==2021 & FO==0')
    .groupby('user_id')
    .agg({'total':'mean', 'use_bonus':'mean'})
    .reset_index()
)
user_group_2021['total_category'] = user_group_2021.apply(total_category, axis=1)
user_group_2021 = (
    user_group_2021.pivot_table(
    index='total_category',
    values=['use_bonus', 'user_id'],
    aggfunc={'use_bonus':'mean', 'user_id':'count'}
    )
)
orders_use_bonus_2021 = orders.query('order_year==2021 & use_bonus == 1')
orders_use_bonus_2021 = (
    orders_use_bonus_2021.groupby('user_id')
                         .agg({'total':'mean', 'pay_bonus':'mean'})
                         .reset_index()
)
orders_use_bonus_2021['total_category'] = orders_use_bonus_2021.apply(total_category, axis=1)
orders_use_bonus_2021 = (
    orders_use_bonus_2021.pivot_table(
        index='total_category', 
        values='pay_bonus',
        aggfunc='mean'
    )
)
user_group_2021 = user_group_2021.merge(
    orders_use_bonus_2021, 
    how='left', 
    on='total_category'
)
user_group_2021 = user_group_2021.rename(
    columns={'use_bonus': 'Пользуются бонусами', 
             'user_id': 'Кол-во клиентов в категории', 
             'pay_bonus':'В среднем тратят бонусов'})
user_group_2021 = user_group_2021.style.format({'Пользуются бонусами': '{:.2%}', 'В среднем тратят бонусов': int})
user_group_2021.to_excel('user_group_2021.xlsx')
print('')
print('Заказы 2021 года')
display(user_group_2021)


Заказы 2021 года


,Пользуются бонусами,Кол-во клиентов в категории,В среднем тратят бонусов
total_category,,,
1000-2000,47.93%,7305,153
2001-3000,52.55%,1919,224
3001-4000,47.47%,503,257
4001-5000,47.42%,128,316
5001-7000,47.13%,79,419
7001-10000,40.00%,19,761
Больше 10000,20.00%,10,871
До 1000,55.26%,1039,182


In [8]:
# Сделаем отчет за 2022 год
user_group_2022 = (
    orders.query('order_year==2022 & FO==0')
    .groupby('user_id')
    .agg({'total':'mean', 'use_bonus':'mean'})
    .reset_index()
)
user_group_2022['total_category'] = user_group_2022.apply(total_category, axis=1)
user_group_2022 = (
    user_group_2022.pivot_table(
    index='total_category',
    values=['use_bonus', 'user_id'],
    aggfunc={'use_bonus':'mean', 'user_id':'count'}
    )
)
orders_use_bonus_2022 = orders.query('order_year==2022 & use_bonus == 1')
orders_use_bonus_2022 = (
    orders_use_bonus_2022.groupby('user_id')
                         .agg({'total':'mean', 'pay_bonus':'mean'})
                         .reset_index()
)
orders_use_bonus_2022['total_category'] = orders_use_bonus_2022.apply(total_category, axis=1)
orders_use_bonus_2022 = (
    orders_use_bonus_2022.pivot_table(
        index='total_category', 
        values='pay_bonus',
        aggfunc='mean'
    )
)
user_group_2022 = user_group_2022.merge(
    orders_use_bonus_2022, 
    how='left', 
    on='total_category'
)
user_group_2022 = user_group_2022.rename(
    columns={'use_bonus': 'Пользуются бонусами', 
             'user_id': 'Кол-во клиентов в категории', 
             'pay_bonus':'В среднем тратят бонусов'})
user_group_2022 = user_group_2022.style.format({'Пользуются бонусами': '{:.2%}', 'В среднем тратят бонусов': int})
user_group_2022.to_excel('user_group_2022.xlsx')
print('')
print('Заказы 2022 года')
display(user_group_2022)


Заказы 2022 года


,Пользуются бонусами,Кол-во клиентов в категории,В среднем тратят бонусов
total_category,,,
1000-2000,43.62%,7730,164
2001-3000,47.24%,2737,246
3001-4000,42.17%,1104,289
4001-5000,48.26%,338,347
5001-7000,49.65%,238,447
7001-10000,35.21%,99,494
Больше 10000,37.12%,28,1736
До 1000,39.08%,1098,215


In [9]:
# Сделаем отчет за август 2022 года
user_group_2022_august = (
    orders.query('order_year==2022 & FO==0 & order_month==8')
    .groupby('user_id')
    .agg({'total':'mean', 'use_bonus':'mean'})
    .reset_index()
)
user_group_2022_august['total_category'] = user_group_2022_august.apply(total_category, axis=1)
user_group_2022_august = (
    user_group_2022_august.pivot_table(
    index='total_category',
    values=['use_bonus', 'user_id'],
    aggfunc={'use_bonus':'mean', 'user_id':'count'}
    )
)
orders_use_bonus_2022_august = orders.query('order_year==2022 & use_bonus==1 & order_month==8')
orders_use_bonus_2022_august = (
    orders_use_bonus_2022_august.groupby('user_id')
                         .agg({'total':'mean', 'pay_bonus':'mean'})
                         .reset_index()
)
orders_use_bonus_2022_august['total_category'] = orders_use_bonus_2022_august.apply(total_category, axis=1)
orders_use_bonus_2022_august = (
    orders_use_bonus_2022_august.pivot_table(
        index='total_category', 
        values='pay_bonus',
        aggfunc='mean'
    )
)
user_group_2022_august = user_group_2022_august.merge(
    orders_use_bonus_2022_august, 
    how='left', 
    on='total_category'
)
user_group_2022_august = user_group_2022_august.rename(
    columns={'use_bonus': 'Пользуются бонусами', 
             'user_id': 'Кол-во клиентов в категории', 
             'pay_bonus':'В среднем тратят бонусов'})
user_group_2022_august = user_group_2022_august.style.format({'Пользуются бонусами': '{:.2%}', 'В среднем тратят бонусов': int})
user_group_2022_august.to_excel('user_group_2022_august.xlsx')
print('')
print('Заказы Август 2022 года')
display(user_group_2022_august)


Заказы Август 2022 года


,Пользуются бонусами,Кол-во клиентов в категории,В среднем тратят бонусов
total_category,,,
1000-2000,49.70%,1207,158
2001-3000,50.36%,414,269
3001-4000,50.00%,220,278
4001-5000,38.89%,45,231
5001-7000,61.54%,39,408
7001-10000,42.86%,21,630
Больше 10000,33.33%,3,685
До 1000,59.48%,174,201
